# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
file = pd.read_csv("cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [5]:
# Heatmap of humidity
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Iterate through 
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -18.1326021,
                    "lng": 13.882402
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.13097267010728,
                        "lng": 13.88388602989272
                    },
                    "southwest": {
                        "lat": -18.13367232989272,
                        "lng": 13.88118637010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "baa79eb1237e8b4d196fc6c2ea4f75a18ac9e9e2",
            "name": "Hotel Pension Le Manoir",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2448,
                    "html_attributions": [
               

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAADbNi5Lvn6l6ZOZfhReLA7D9MaRY1fEIEgWioBgJH1AQ4lfpc9T4UctyaI74sY11RG4dMAcyTsQ8KzDT--HCNZSB0MhfdXEw5fKr_CLuomQwz1H0mq1fddMbARjea75Rgn0ewuS6qn8gE10bs4c5TLAnBnyjepmk3aNaeHvQFzdJvk-gC1ziGloha3KvisAHRgFhwQjK7pRkBS66Vh_AVwFDiYQ1mf5rjpA7QJq70ENsPTcK39eZq978SsW7WGUCD0U4Ddh4wF2sqTfUTsI1cZWvwDV62bEVHgcC_5xTmYxXwwXKGS-0yBfLQHCd34HMkD6x89HuJhigCNkEUpt32JgF8aTsC5FyPROTpzliF4x7hDXLdYe3evmP77X_pcMuw53C9V5-rgocBLFSeQda9j4SEEmj4aWOB_4gxUbgIIOK_K4aFLcWQIJYBh4AbaM6F6D1y9phlxUz",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.6346813,
                    "lng": -46.0736289
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.63314217010728,
                        "lng": -46.07214152010727
                    },
                    "southwest": {
                        "lat": -20.63584182989272,
                  

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAMPTkmWunUc9I3ig5EdLepcRaIgPUQKVs1J0dwU2R28a0l104BLVCnJs1_5afQUe2hQ_v2AoBQgr_05WTHEw9DOK12_c_RPQxxGePRnnpOsBmG9PAg15sIMMawkabvXoLgup7C_B0gMArIPsT5OYKnuPjwY35PPlGKf2uWLHC0MtH2Nu7CsPqwvTaX6jKRXZ1IyAkNobRGho9d5z-KA5AjwpPoV7nzoPKRD4jRXtCW5A65x4tny0HV6VBDVUcRGcY5ksNMNMn5NAMaMyyVA5LvRAIMlKsMsKOQV5ZyPJGhReSFCDCQDZ4IM4-A3bfWsWi8Y_DCQsAhbwJJ8Hd6L95pZLPWNZVEpwaS_rt4Mx8iJoqVSUUrBZoun1JTIgRu5S8Djq3VcEl4i99wDUdr3Z6bISEHgHmUlCcc5mCaIjlG8sl-QaFE6ex-cmG45dstvDcGgKEuqpLXZb",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -29.7609127,
                    "lng": -50.0140734
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.75972052010728,
                        "lng": -50.01257622010728
                    },
                    "southwest": {
                        "lat": -29.76242017989272,
                  

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAD9pBnf7mdaLzg_XueTmVpP4I_jlNc5zng1kohyNk_2Om-NbwJ1jwghYoadD-veaDZm8N8UAdha5h-pW6lf8GWKMMNuBWSX96piJSPBofi2K1IS5UhB6pQm3MSjnvBT3LohGQ8uQySCu32J0set7LicR5h8e1rCwILi6QQWbIuai2AvdBWaxc44d7GVDRA4N7nybFDqKR_rcZa7ELsNkxXVBjipZx7QLGZ0OlZ00QcYZ5-okAU61vPNYIEn8zQp8zJh0mHa-DxYw4USfLDMeuvc9VqH_vFr4e0ZovPCG-4vvuQilTxxEVzmqtgPbVJ65JCCKrsAQakK-0DEu9da8UjyGxpxxK-mT4aS4n2RAf-poF6Zxag85LmwnjG23VOAIkhb_bBfCTjd17VmD5X2JaCwSEIrsrkOPa43F24rQhVt76UYaFLsBvrU-wKfQF-uCWc3CoG36t3la",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.290218,
                    "lng": 44.299996
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.28947867010728,
                        "lng": 44.30136652989273
                    },
                    "southwest": {
                        "lat": -20.29217832989272,
                      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 15.0040551,
                    "lng": 74.032279
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.00535382989272,
                        "lng": 74.03354577989272
                    },
                    "southwest": {
                        "lat": 15.00265417010728,
                        "lng": 74.03084612010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "7481f733341b77b9956a3badfe6ac2b68e6e8122",
            "name": "Oceanic Hotel",
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/113116447481431456013/photos\">Ocea

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAPgLp5ynKJgWdsPxvjvUiBUFkTKqGpAgX_0pucAD_Ul1maZGZmsgTICKQtbjw0kRG_HIo2aRevoFRvdcyy9BLf0Wb0Vt5dux_Likd_PuP7cVnGF2E4ClYt8wrqURBPEDuGIWL64mBAYayXhOSr3EHT1lunWo1wUPXrScWGl3mtDlJ5Wci6a534cM7TqxfDmLgDoZTd3pAtuFDtSYju56lyEKe-r-m3w7YhCzMuVBUsuW465szvW-Or3Tpk1k7QgstnG6UTS75RIyT00H4STXs38u9-o3qUuV4GgNDnOrlHnTg6Hi2JllU3Y-hLi4XDcmKTVmg_EOaO65Vn3HDtVlTel-CNyZdRDpb-VYtL7OyKovtgoTY-2YAjvZyeOURYWNqokP7lzOzSVdhpa5zxF_DKsSEB5blevVNc2Dsd_6ZMLyz5waFHIMQM6PVW-CzEPjo83DONdTEnQz",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 35.1782726,
                    "lng": -2.9221445
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.17954257989272,
                        "lng": -2.920968020107277
                    },
                    "southwest": {
                        "lat": 35.17684292010728,
                      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.9164309,
                    "lng": 96.4563258
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.9177379,
                        "lng": 96.45768732989272
                    },
                    "southwest": {
                        "lat": 22.9125099,
                        "lng": 96.45498767010729
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f63499003fa624669e9c027aebacd2f2a4a8d167",
            "name": "Golden Butterfly Hotel",
            "photos": [
                {
                    "height": 4000,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/104812487407125106787/photos\">Thant Zi

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAFZoZi_0IPRI8XlJzgAAHj7V3EC7_gvke_y1WMHU0RWxI6bPgVXgondUN0km3zLURhCxHhD3cMoTXNgfs26cSz2bM4rN3Nu3DwDVFg9nha8lqBZiO9G-KZtKB91dgKJyP8Io4XUOvfA3pe-ZH_7plK-7XsVBUpZL7ylb3q4pGjYqoleOoWr9pIcIEsePTzTNinmdoPfaNh1Pm42J4vOpDz-G86RoedulzDbQLAFth1iUypURT7dMjarG5uRdl3KhTuD7BnfMUouV8CRNpUHG9V2AydGSfaGoQosoYN9QzJ3NNk1KzivrdeMINkT-4cnonML1AdujjJArmPhIryTsXe6zXhgUipvmdrAFnwwEtNWuPeIClwjJ7zlMc9QgRO3J8tO8_UaFSN0UG1IvSZasHWoSEERR5p5pN1D1PGShHcZicnIaFNJ8u-TxT_Ok3sw3WEmHpq_X45j0",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.8802013,
                    "lng": 59.2162932
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88151887989271,
                        "lng": 59.21771767989272
                    },
                    "southwest": {
                        "lat": 32.87881922010727,
                       

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Hotel Pension Le Manoir
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Engenho da Serra
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Bassani
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Hotel maeva morondava
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Oceanic Hotel
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Golden Butterfly Hotel
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,هتل پرويز


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))